# Lecture 3 plots and analysis

In [ ]:
from pysolar import solar
from datetime import datetime, timezone, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from IPython.display import display, Markdown
import sys
from enn554.sun import compute_solar_angles, sun_path_diagram, black_body_spectral_radiance, plane_of_array_irradiance, sun_path_3d
from enn554.utilities import read_TMY

## Sun-earth distance

In [ ]:
r0 = 1.495e11
sind = lambda x: np.sin(np.deg2rad(x))
cosd = lambda x: np.cos(np.deg2rad(x))
t = np.arange('2023-01-01','2023-12-31',dtype='datetime64[D]')
doy = (t-t[0].astype('timedelta64[D]')).astype('float64')
sun_earth_distance = lambda x: r0*(1+0.017*sind(360*(x-93)/365.0))
sed = sun_earth_distance(doy)

fig,ax = plt.subplots()
ax.plot(t,sed)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
xlims = ax.get_xlim()
ylims = ax.get_ylim()
ax.grid('on')

# extrema
doy_max_distance = 90*365/360+93
ymax = sun_earth_distance(doy_max_distance)
xmax = t[np.argmin(np.abs(doy-doy_max_distance))]

xmaxn = (mdates.date2num(xmax)-xlims[0])/np.diff(xlims) # put into axis coordinates
ymaxn = (ymax-ylims[0])/np.diff(ylims)
ax.axhline(y=ymax,xmax=xmaxn[0],ls=':',color='red')
ax.axvline(x=xmax,ymax=ymaxn[0],ls=':',color='red')

doy_min_distance = -90*365/360+93
ymin = sun_earth_distance(doy_min_distance)
xmin = t[np.argmin(np.abs(doy-doy_min_distance))]
xminn = (mdates.date2num(xmin)-xlims[0])/np.diff(xlims) # put into axis coordinates
yminn = (ymin-ylims[0])/np.diff(ylims) # put into axis coordinates
ax.axhline(y=ymax,xmax=xminn[0],ls=':',color='red')
ax.axvline(x=xmin,ymax=yminn[0],ls=':',color='red')

ax.set_title("Sun-Earth Distance")
print(f"Maximum: {ymax:.2e} (on {xmax})")
print(f"Minimum: {ymin:.2e} (on {xmin})")


## Solar specturm — approximation of the sun as a black body

The below code demonstrates the use of Planck's Law to approximate the solar spectrum and compute different quantities of interest. 

Plot the blackbody spectrum and compute:
1) the total irradiation
2) the solar irradiation in the visible spectrum

In [ ]:
T_sun = 5800.0 # K
Ω_sun = 68e-6  # [sr], solid angle of sun seen by earth, see [1] or below
λ = np.arange(0.1,4.0,0.001) # [µm]
I_sun = Ω_sun * black_body_spectral_radiance(λ,T_sun,display_eqn=True)
# I_sun = lambda x: C1 / (x**5 * (np.exp(C2/x/T_sun)-1) ) # [W/m2/µm]
# E_sun = lambda x: 3.7419e8 / (x**5 * (np.exp(14400/x/T_sun)-1) ) # [W/m2/µm] THIS IS SPECTRAL RADIANT EXITANCE, not spectral radiance
# E_sun = lambda x: 8105/ (x**5 * (np.exp(2.480/x)-1) ) # [W/m2/µm]

# NREL AM0
df = pd.read_excel("data/astmg173.xls",sheet_name="SMARTS2",index_col=0,skiprows=1)
df *= 1000
df.index /= 1000

fig,ax = plt.subplots()
df.plot(ax=ax,y="Etr W*m-2*nm-1",label='AM0 (ASTM G173-03)')
ax.plot(λ,I_sun,label='Black body',linewidth=3)
ax.legend()
ax.set_xlabel(r'Wavelength, $\lambda$ [µm]')
ax.set_ylabel(r'Spectral Irradiance, $G_{sc,\lambda}$($\lambda$) [W/$m^2$/µm]')
ax.set_ylim((0,2200))

Gsc = np.trapz(I_sun,x=λ) 
total_exitance = np.trapz(np.pi/Ω_sun * I_sun,x=λ) 
print(f"Total irradiation from black body: {Gsc:.2f} (W/m2)")
idx = np.where((λ>=0.380) & (λ<=0.780))[0]
λ_visible = λ[idx]
I_sun_visible = I_sun[idx]
G_visible = np.trapz(I_sun_visible,x=λ_visible)
print(f"Total irradiation in  ({λ_visible[0]:.2f},{λ_visible[-1]:.2f}) body: {G_visible:.2f} W/m2 ({100*G_visible/Gsc:.1f} %)")
print(f"Total exitance (MW/m2):  {total_exitance/1e6:.2f} (numerical inegration), {5.670e-8*T_sun**4 / 1e6:.2f} (Stefan-Boltzman)")

display(Markdown( rf""" $I(\lambda) = \frac{{{1.1911e8*Ω_sun:.0f}}}{{\lambda^5 \left[exp\left(\frac{{{1.4384e4/T_sun:.4f}}}{{\lambda}}\right) -1\right]}} \qquad [W/m^2 \, \mu m]$
                    """))

## Solar constant

In [ ]:
I_sun = 6.33e7
R_sun = 1.39e9/2.0
r_0 = 1.495e11
G_sc = I_sun * 4*np.pi*R_sun**2/(4*np.pi*r_0**2)
print(f"The solar irradiance on earth is (on average) G_sc = {G_sc:.2f} W/m^2")

Comparison of the two methods of going from radiance to extraterrestrial irradiance

In [ ]:
radiance_numerator = 1.1911e+08
print(f"C1 using exitance: {R_sun**2/r_0**2*np.pi*radiance_numerator}")
print(f"C1 using Omega_sun: {radiance_numerator*Ω_sun}")

### Aside: solid angle of sun seen from earth and vice versa

In [ ]:
θ = 32/60.0/2.0 # 32' solar angle from [2]
Ω_sun_1 = 2*np.pi*(1-cosd(θ)) # conical solid angle
Ω_sun_2 = 4*np.pi*sind(θ/2)**2 # alternative expression for the above. 
Ω_sun = 2*np.pi*(1-np.sqrt( (r0**2-R_sun**2)/r0**2 ))
print(f"The solid angle of the sun (as seen from Earth) is about {Ω_sun*1e6:.2f} µsr")
Ω_sun_1, Ω_sun, Ω_sun_2  # first method is from Chapter 2 in [5]

In [ ]:
θ = 17/3600*np.pi/180/2.0 # 17'' solar angle from [2]
Ω_earth = 2*np.pi*(1-np.cos(θ)) # conical solid angle
print(f"The solid angle of the earth (as seen from a the sun) is about {Ω_earth*1e9:.2f} nsr")

## Sun angles
Watch out for:
* Different radian/degree conventions with sines and cosines. I'm using degrees for this content
* Different conventions for longitude. Pysolar uses -180 to 180 while the equations from Duffie and Beckman use 0 to 360 (increasing going westward).

In [ ]:
# Brisbane, Australia (27.4705 S, 153.0260 E)
lat = -27.47
lon_st = 360-150 # E -> subtract from 360
lon_pysolar = 153.02
lon = 360-153.0260 # E -> subtract from 360
utc_offset = 10 # hours

# # Madison, Wisconsin for example 1.6.2a in Duffie and Beckman
# lat = 43.0
# lon = 89.4
# lon_pysolar = -89.4
# lon_st = 90
# utc_offset = -6 # hours

# local time
s = 0
m = 30 # should make t_solar = 9.5
h = 5
day = 1
month = 1
year = 2014

dt = datetime(year,month,day,h,m,s,tzinfo=timezone(timedelta(hours=utc_offset)))
doy = dt.timetuple().tm_yday # day of year
γ_s_pysolar,θ_alt_pysolar = solar.get_position(lat,lon_pysolar,dt) 
θ_z_pysolar = 90-θ_alt_pysolar
γ_s_pysolar -= 180 # pysolar uses 0 at north and positive angles east of north up to 360 https://pysolar.readthedocs.io/en/latest/#examples

sun_angles = compute_solar_angles(dt,lat,lon,lon_st,force_south_as_zero=False)
sun_angles

In [ ]:
print(f"Pysolar: (θ_z,γ_s), West of South: ({θ_z_pysolar:.2f},{γ_s_pysolar:.2f})")
print(f"Calcs: (θ_z,γ_s), West of {sun_angles['azimuth_zero']}: ({sun_angles['zenith']:.2f},{sun_angles['azimuth']:.2f})")

Remaining differences seem to be due to some extra corrections (e.g. refraction corrections) applied by Pysolar

## Sun Paths for different locations

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

location_name = "Brisbane, Australia"
lat = -27.4705
lon_st = 360-150 # E -> subtract from 360
lon_pysolar = 153.0260
lon = 360-153.0260 # E -> subtract from 360
utc_offset = 10

# location_name = "Madison, Wisconsin, USA"
# lat = 43.0
# lon = 89.4
# lon_st = 90
# utc_offset = -6 # hours

# location_name = "Tromsø, Norway"
# lat = 69.6492
# lon = 360-18.9560
# lon_st = 360-15
# utc_offset = 1 # hours

summer_solstice = datetime(2024,6,20)
winter_solstice = datetime(2024,12,21)
dts = [summer_solstice,winter_solstice]
dts_of_interest = np.array([dt.timetuple().tm_yday for dt in dts],dtype=np.float64)  # day of year
doy_grid = np.arange(0,366,36,dtype=np.float64)

for d in dts_of_interest:
    if np.abs(doy_grid-d).min() < 10:   
        idx = np.abs(doy_grid-d).argmin()
        doy_grid = np.delete(doy_grid,idx)
    doy_grid = np.r_[doy_grid,d]
doy_grid = np.sort(doy_grid)

hod_grid = np.arange(3,22,0.1)
year = datetime.now().year
dt0 = datetime(year,1,1,0,0,0,tzinfo=timezone(timedelta(hours=utc_offset)))
dat = sun_path_diagram(dt0,hod_grid,doy_grid,lat,lon,lon_st,force_south_as_zero=False,location_name=location_name)

In [ ]:
# Create a figure
import matplotlib.dates as mdates
fig = plt.figure(figsize=(10,12))
ax = fig.add_subplot(111, projection='3d')
azimuths = dat['data']['azimuth']
zeniths = dat['data']['zenith']
azimuth_zero = dat['data']['azimuth_zero']
X,Y,Z = azimuths,np.repeat(doy_grid[:,np.newaxis],azimuths.shape[1],axis=1),90-zeniths
X[Z<0]=np.nan
Y[Z<0]=np.nan
Z[Z<0]=np.nan
ax.plot_surface(X,Y,Z)

ax.set_zlim((0,90))
ax.set_ylim((0,366))
ax.yaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
ax.set_xlim((-180,180))
ax.set_xlabel(f"Azimuth Angle (degrees, + is West of {azimuth_zero})")
# ax.set_ylabel("Day of Year")

ax.set_zlabel("Altitude angle (degrees)")
ax.set_title(location_name)

In [ ]:
interactive = sun_path_3d(dt0,hod_grid,doy_grid,lat,lon,lon_st,location_name=location_name,convention='sam')
# html_str = mpld3.fig_to_html(fig)
# with open("outputs/interactive_plot.html", "w") as f:
#     f.write(html_str)

## Computing the annual insolation for Brisbane from a TMY file

Load in the NREL TMY file and specify location information. I'm using a location that's closer to the C-block roof than that listed in the TMY file

In [ ]:
tmy,metadata = read_TMY('data/brisbane_tmy.csv')
L_tz = 360-150 # E -> subtract from 360
L = 360-153.0260 # E -> subtract from 360
φ = -27.4705

Angles of collector (azimuth is from S)

In [ ]:
β,γ = 22.2518924961955,180
ground_albedo = 0.0

Compute the POA irradiance and sum to get insolation in kWh (since time steps are one hour). Note that I have to ignore the leapday because the TMY file will not be a leap year, but some of the "typical" conditions are taken from leap years. 

In [ ]:
irradiance = []
az = []
zen = []
st = []
for ii in range(tmy.shape[0]):
    DNI,GHI = tmy.DNI[ii],tmy.GHI[ii]
    t = tmy.Datetimes[ii]
    G,sp = plane_of_array_irradiance(t,DNI,GHI,φ,L,L_tz,β,γ,ρ=ground_albedo,model="ISM",θ_z_max=90,
                                  clip_negative=True,ignore_leapday=True)
    irradiance.append(G)
    az.append(sp['azimuth'])
    zen.append(sp['zenith'])
    st.append(sp['solar_time'])

print(f"Insolation is: {sum(irradiance)/1000:.2f} kWh/m2/year")

df = pd.DataFrame({'Azimuth':az,'Zenith':zen,'POA Irradiance':irradiance,'solar_time':st})
df.to_excel("../lectures/Lecture 3 MMPE - solar radiation fundamentals/python_results.xlsx")

Compare with excel calcs

In [ ]:
dat = pd.read_excel("../tutorials/Solar Irradiance/poa_tutorial_solutions.xlsx",
                    sheet_name="POA_NO_SHADE",skiprows=[0,1],usecols=["POA (ISM) zero corrected"]).values
fig,ax = plt.subplots()
ax.plot(dat.flatten()-np.array(irradiance))

# References
 1. [TU Delft Notes](https://ocw.tudelft.nl/wp-content/uploads/solar_energy_section_5_Appendix.pdf)
 2. N. B. John A. Duffie, William A. Beckman, Solar Engineering of Thermal Processes, Photovoltaics and Wind, Fifth. Hoboken, New Jersey: John Wiley & Sons, 2020.
 3. G. M. Masters, Renewable and Efficient Electric Power Systems. Newark, UNITED STATES: John Wiley & Sons, Incorporated, 2013.
 4. M. Kanoğlu, Y. A. Çengel, and J. M. Cimbala, Fundamentals and Applications of Renewable Energy, 2nd Edition. New York: McGraw-Hill Education, 2023.
 5. K. Lovegrove and W. Stein, Eds., Concentrating solar power technology. Woodhead Publishing. 


